In [0]:
%sql
-- add columns to our table
ALTER TABLE workspace.csv.bitcoin_history_data
ADD COLUMN Range DOUBLE, PriceGap DOUBLE

In [0]:
%sql
-- fill new columns
UPDATE workspace.csv.bitcoin_history_data
SET
  Range = ROUND(ABS(High - Low), 2),
  PriceGap = ROUND(ABS(Open - Close), 2),
  High = ROUND(High, 2),
  Low = ROUND(Low, 2),
  Open = ROUND(Open, 2),
  Close = ROUND(Close, 2);

In [0]:
from pyspark.sql.functions import to_date, col, date_format
import pandas as pd

# load SQL table into Spark data frame
df = spark.table('workspace.csv.bitcoin_history_data')
# Spark operations are lazy; they don’t execute until needed. This allows for efficient filtering, joins, 
# transforms before pulling it into Pandas

# edit Date column format
df = df.withColumn("DateFormatted", date_format("Date", "MM-dd-yyyy"))

# convert to pandas
pdf = df.toPandas()

# create target column from close column shifted up by one
# doing this makes each columns' target the next day's price. This is how we teach the model to predict price
pdf['Target'] = pdf['Close'].shift(-1)

# for now, we dont want an empy row at the end so we drop it
pdf.dropna(inplace=True)

# sort by date
pdf = pdf.sort_values(by='Date')

#convert Date column into datetime values
pdf['Date'] = pd.to_datetime(pdf['Date'])

# add day of week columns, day name and numerical value
pdf['DayOfWeek'] = pdf['Date'].dt.day_name()
pdf['DayOfWeekNum'] = pdf['Date'].dt.weekday

display(pdf)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# features (input values for Model)
features = ['Open','Close','Volume','DayOfWeekNum']
x = pdf[features]

# target (output value we want the model to predict)
y = pdf['Target']

# split data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(
    x,y,
    # for this model we are using dates as the index, so we want to keep the order
    shuffle=False,
    # test set will be 20% of the data, training will be the other 80%
    test_size=0.2
)

# train the model
LR_model_1 = LinearRegression()
LR_model_1.fit(x_train, y_train)

# model Prediction
predictions = LR_model_1.predict(x_test)


In [0]:
import pandas as pd

# reset the index on test set to avoid misaligned rows/cols
x_test_reset = x_test.reset_index(drop=True)
y_test_reset = y_test.reset_index(drop=True)

# extract the date values for the test-set rows from the original dataframe,
# thenreset the index so the dates align 1-to-1 with x_test after splitting.
date_col = pdf.loc[x_test.index, 'Date'].reset_index(drop=True)

# build results table
results = pd.DataFrame({
    'Date': date_col,
    'Close': x_test_reset['Close'].round(2),
    'Predicted_Close': predictions.round(2)
})

# to results table, add col for difference between price and predicted price 
results['difference'] = (results['predicted_price'] - results['price']).round(2)
# add col for margin of percenterror percent 
results['% Error'] = ((results['price'] - results['predicted_price']).abs() / results['price'] * 100).round(2)

# show results
display(results)


